# EasyLLM: Simplified OpenAI API Abstraction

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://www.freeiconspng.com/thumbs/calculator-icon/calculator-icon-26.jpg"> 
</p>

</div>

## Description:
This app provides an easy-to-use abstraction for the OpenAI API. It simplifies generating text and objects using OpenAI's GPT models with customizable prompts, making it ideal for tasks like content creation, generating structured data, and more. The app includes setup for dependencies, environment variables, and example workflows for generating text and structured responses.

---

## Step 1: Boilerplate Setup

This step sets up the boilerplate code for the project. It includes:

- ### Import Statements:

    - `os`: Used for running shell commands and accessing environment variables.

    - `load_dotenv`: Loads environment variables from a `.env` file into the system for easy configuration.

    - `clear_output`: Clears the notebook's output to keep it clean after successful setup.



- ### Global Variables:

    - `requirements_installed`: Tracks whether dependencies are already installed.
    
    - `max_retries`: Limits how many times the code will retry installing dependencies in case of failure.
    
    - `REQUIRED_ENV_VARS`: Specifies the environment variables that must be present.



- ### `install_requirements` Function:

    - Uses the `os.system` command to run `pip install -r requirements.txt`.

    - If the installation fails, it retries up to `max_retries`.

    - If retries are exhausted, it exits the program with a failure code.



- ### `setup_env` Function:

    - Loads environment variables from `.env`.

    - Verifies the presence of each required variable using the `check_env` function.

    - Exits the program if any required environment variable is missing.



- ### Execution:

    - Calls `install_requirements` to install dependencies.

    - Calls `setup_env` to validate the environment.

    - Clears the output and confirms the setup is complete.



In [ ]:
# Boilerplate: This block goes into every notebook.
# It sets up the environment, installs the requirements, and checks for the required environment variables.

from IPython.display import clear_output
from dotenv import load_dotenv
import os

requirements_installed = False
max_retries = 3
retries = 0
REQUIRED_ENV_VARS = ["OPENAI_API_KEY"]


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return


def setup_env():
    """Sets up the environment variables"""

    def check_env(env_var):
        value = os.getenv(env_var)
        if value is None:
            print(f"Please set the {env_var} environment variable.")
            exit(1)
        else:
            print(f"{env_var} is set.")

    load_dotenv()

    variables_to_check = REQUIRED_ENV_VARS

    for var in variables_to_check:
        check_env(var)


install_requirements()
clear_output()
setup_env()
print("🚀 Setup complete. Continue to the next cell.")

## Step 2: Defining the EasyLLM Class:

### **Purpose**  
The `EasyLLM` class provides an abstraction layer for interacting with OpenAI's API. It simplifies the process of:  

- Generating text responses.  
- Parsing structured responses into objects using Pydantic models.  
- Switching between OpenAI models.  

---

### **Detailed Code Explanation**

#### **Class-Level Defaults**

- `DEFAULT_OPENAI_MODEL`: Sets the default OpenAI model to `gpt-4o-mini`.  

- `DEFAULT_SYSTEM_PROMPT`: Provides a default system instruction for the AI assistant.  

- `DEFAULT_TEMPERATURE` and `DEFAULT_MAX_TOKENS`: Control the response style (creativity) and length.  

---

#### **`__init__` Method**  

- Accepts `api_key`, `model`, and logging preferences (`verbose` and `debug`).  

- Initializes the OpenAI client with the provided API key.  

- Prints initialization messages if `verbose` or `debug` mode is enabled.  

---

#### **`generate_text` Method**  

- Takes a prompt and optional parameters like `system_prompt`, `temperature`, and `token_limit`.  

- Sends a request to OpenAI's `chat.completions.create` API.  

- Logs detailed request parameters if `debug` is enabled.  

- Extracts and returns the content from the API's response.  

---

#### **Error Handling**  

- Catches exceptions, prints error messages, and optionally logs stack traces in `debug` mode.  

---

#### **`generate_object` Method**  

- Extends `generate_text` to parse responses into structured objects using Pydantic models.  

- Takes a `response_model` (a Pydantic model class) to define the expected response format.  

- Returns the parsed object if successful.  

---

#### **Model Management**  

- **`get_model`**: Returns the currently active model.  

- **`set_model`**: Changes the model used for generating responses.  


In [2]:
import os
import openai
from pydantic import BaseModel
import traceback
from typing import Union
import json

DEFAULT_OPENAI_MODEL = "gpt-4o-mini"
DEFAULT_SYSTEM_PROMPT = "You are an intelligent AI assistant. The user will give you a prompt, respond appropriately."
DEFAULT_TEMPERATURE = 0.5
DEFAULT_MAX_TOKENS = 1024


class EasyLLM:
    """
    A simple abstraction for the OpenAI API. It provides easy-to-use methods to generate text and objects using the OpenAI API.
    A demonstration for the "How to build an Abstaction with Open AI API" blog post.
    Author: Aditya Patange (AdiPat)
    """

    def __init__(
        self,
        api_key=os.getenv("OPENAI_API_KEY"),
        model=DEFAULT_OPENAI_MODEL,
        verbose=True,
        debug=True,
    ):
        self.verbose = verbose
        self.debug = debug

        if self.verbose:
            print("EasyLLM: Powering up! 🚀")

        self.api_key = api_key
        self.openai = openai.OpenAI(api_key=api_key)
        self.model = model

        if self.verbose:
            print(f"EasyLLM: Model set to {model}.")
            print("EasyLLM: Ready for some Generative AI action! ⚡️")

    def generate_text(
        self,
        prompt: str,
        system=DEFAULT_SYSTEM_PROMPT,
        temperature=DEFAULT_TEMPERATURE,
        max_tokens=DEFAULT_MAX_TOKENS,
    ) -> Union[str, None]:
        """Generates text using the OpenAI API."""
        try:
            if self.verbose or self.debug:
                print(f"Generating text for prompt: {prompt}")

            if self.debug:
                params = {
                    "prompt": prompt,
                    "system": system,
                    "temperature": temperature,
                    "max_tokens": max_tokens,
                    "model": self.model,
                }
                params = json.dumps(params, indent=2)
                print(f"Params: {params}")
            response = self.openai.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system},
                    {"role": "user", "content": prompt},
                ],
                temperature=temperature,
                max_tokens=max_tokens,
            )

            response = response.choices[0].message.content

            if self.verbose or self.debug:
                print("Text generated successfully. 🎉")

            if self.debug:
                response = json.dumps(response)
                print(f"EasyLLM Response: {response}")
            return response
        except Exception as e:
            print(f"Failed to generate text. Error: {str(e)}")
            if self.debug:
                traceback.print_exc()
            return None

    def generate_object(
        self,
        prompt: str,
        response_model: BaseModel,
        system=DEFAULT_SYSTEM_PROMPT,
        temperature=DEFAULT_TEMPERATURE,
        max_tokens=DEFAULT_MAX_TOKENS,
    ) -> Union[BaseModel, None]:
        """Generates an object using the OpenAI API and given response model."""
        try:
            if self.verbose or self.debug:
                print(f"Generating object for prompt: {prompt}")

            if self.debug:
                params = {
                    "prompt": prompt,
                    "system": system,
                    "temperature": temperature,
                    "max_tokens": max_tokens,
                    "model": self.model,
                }
                params = json.dumps(params, indent=2)
                print(f"Params: {params}")

            response = self.openai.beta.chat.completions.parse(
                messages=[
                    {"role": "system", "content": system},
                    {"role": "user", "content": prompt},
                ],
                response_format=response_model,
                model=self.model,
                temperature=temperature,
                max_tokens=max_tokens,
            )

            if self.verbose or self.debug:
                print("Object generated successfully. 🎉")

            if self.debug:
                response_json = response.model_dump_json()
                print(f"EasyLLM Response: {response_json}")
            return response.choices[0].message.parsed
        except Exception as e:
            print(f"Failed to generate object. Error: {str(e)}")
            if self.debug:
                traceback.print_exc()
            return None

    def get_model(self) -> str:
        """Gets the current model."""
        return self.model

    def set_model(self, model: str) -> None:
        """Sets the model to the given model."""
        try:
            if self.verbose or self.debug:
                print(f"Setting model to {model}")
            self.openai = openai.OpenAI(api_key=self.api_key)
            self.model = model
            if self.verbose or self.debug:
                print(f"Model set to {model}")
        except Exception as e:
            print(f"Failed to set model.\nError: {str(e)}")
            if self.debug:
                traceback.print_exc()
            return None

## Step 3: Generate a Poem Using the EasyLLM Class

### **Purpose**  

This part demonstrates how to use the `EasyLLM` class to generate a simple poem for a specific audience (children aged 15-18) using the `generate_text` method.

---

### **Code Breakdown**

#### **Imports:**

- `pprint (pp)`: Pretty prints Python data structures, which helps in making the output more readable.  

- `clear_output`: Clears the notebook output area after executing the code (often used in Jupyter notebooks).  

---

**Create EasyLLM Instance:** 
``` bash 
llm = EasyLLM() 
```
**Prompt Generation:**
``` bash
prompt = "Generate a simple poem on learning for children between age 15 to 18."
```

The prompt provided to the model asks it to generate a poem tailored for children aged 15 to 18 about learning. This prompt is essential for guiding the model's output

---

```bash
response = llm.generate_text(prompt)
```
This invokes the generate_text method, passing the prompt. The method will interact with OpenAI's API and return the generated text (in this case, a poem). The response is stored in the variable response.

  - clear_output(): Clears any previous outputs in the notebook to keep things neat.

  - pp(f"Prompt: {prompt}"): Pretty prints the original prompt.

  - pp(response): Pretty prints the generated poem. This makes it easier to read and understand the result.

In [ ]:
from pprint import pp
from IPython.display import clear_output

llm = EasyLLM()

prompt = "Generate a simple poem on learning for children between age 15 to 18."

response = llm.generate_text(prompt)
clear_output()
pp(f"Prompt: {prompt}")
pp(response)

# Step 4: Generate a Quote Using Structured Data (Object)

### **Purpose**  

This step demonstrates how to generate a structured object using OpenAI’s API, where the response is parsed into a predefined Pydantic model. The `generate_object` method is used here.

---

### **Code Breakdown**

#### **Imports:**

- Same as before, `pprint` and `clear_output` are imported for output formatting.  

- `BaseModel` (from Pydantic): Used to define response models that are structured and validated.  

- `Enum`: Defines a set of possible values for certain fields (e.g., Emotion and Sentiment).  

- `List`: Allows the use of lists in type annotations.


---

#### **Define the Response Model (QuoteResponse):**

##### **Enum Classes:**

- `Emotion`: This defines possible emotional states associated with a quote (e.g., `HAPPY`, `SAD`, `NEUTRAL`).  

- `Sentiment`: This defines the possible sentiments (e.g., `POSITIVE`, `NEGATIVE`, `NEUTRAL`) that the generated quote might have.  


##### **Tag Class:**

- A `Tag` class is defined, which has two fields:

  - `key`: A string identifier for the tag.

  - `name`: A descriptive name for the tag.


##### **QuoteResponse Class:**

This class models the response that the `generate_object` method will return. It consists of:

- `quote`: The generated quote (a string).

- `meaning`: Explanation of the quote (a string).

- `fictious_author`: A made-up author name for the quote (a string).

- `emotion`: A list of emotions related to the quote.

- `sentiment`: Sentiment of the quote.

- `target_audience`: List of target audiences for the quote (e.g., "students", "teachers").

- `tags`: A list of tags that categorize the quote.

In [ ]:
from pprint import pp
from IPython.display import clear_output
from pydantic import BaseModel
from enum import Enum
from typing import List


class Emotion(str, Enum):
    HAPPY = "happy"
    SAD = "sad"
    NEUTRAL = "neutral"


class Sentiment(str, Enum):
    POSITIVE = "positive"
    NEGATIVE = "negative"
    NEUTRAL = "neutral"


class Tag(BaseModel):
    key: str
    name: str


class QuoteResponse(BaseModel):
    quote: str
    meaning: str
    fictious_author: str
    emotion: List[Emotion]
    sentiment: Sentiment
    target_audience: List[str]
    tags: List[Tag]


llm = EasyLLM()

prompt = "Generate a unique quote on time."

response = llm.generate_object(prompt, response_model=QuoteResponse)
clear_output()
pp(f"Prompt: {prompt}")
pp(response.model_dump_json())

## Conclusion:

This app demonstrates a powerful and flexible way to interact with OpenAI's GPT-4 model using a simple abstraction, EasyLLM. By encapsulating the complexity of direct API calls, this app allows users to easily generate dynamic text responses (like poems or quotes) while also offering the ability to handle more structured outputs (like parsed objects) using Pydantic models.

### Key takeaways from this app include:

- `Simplified API Interaction`: The EasyLLM class abstracts the complexity of interacting with OpenAI’s API, enabling easier use of language models for text generation and structured object creation.

- `Customizable Prompts`: Users can tailor the prompts to generate specific types of content (e.g., poems, quotes) based on their needs.

- `Structured Responses`: For use cases that require structured data, the app demonstrates how to generate and parse complex responses using Pydantic models. This ensures that the output is both predictable and validated.

- `Error Handling and Debugging`: With built-in error handling and debugging options, users can troubleshoot and improve the app’s stability.

This app serves as a solid foundation for developing more advanced AI-powered applications, including those that require natural language processing, sentiment analysis, and content generation. By leveraging the OpenAI API with clear abstractions and simple configurations, this app allows for rapid prototyping of intelligent systems.

---

# Thank You for visiting The Hackers Playbook! 🌐

If you liked this research material;

- [Subscribe to our newsletter.](https://thehackersplaybook.substack.com)

- [Follow us on LinkedIn.](https://www.linkedin.com/company/the-hackers-playbook/)

- [Leave a star on our GitHub.](https://www.github.com/thehackersplaybook)

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
</div>